In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(1234)

In [3]:
total_informative_features = 8
total_shit_features = 4
total_redundant_features = 4

X = np.random.rand(total_informative_features, 50_000)
for i, cardinality in enumerate([2, 3]):
    X[i] = np.round(X[i] * cardinality * 10) % cardinality
    print(i, np.unique(X[i], return_counts=True))

for i in range(2, total_informative_features):
    X[i] = X[i] * np.random.randint(1, 10)

ys = []


def fix_row(row, pairs):
    row = row.copy()
    for fst, snd in pairs:
        fst += total_shit_features
        snd += total_shit_features
        row[fst], row[snd] = row[snd], row[fst]
    return row


def fix_dataset(X, pairs, truncate_informative=True):
    res = np.apply_along_axis(lambda x: fix_row(x, pairs), 1, X)
    if truncate_informative:
        return res[
            :, total_shit_features : total_shit_features + total_informative_features
        ]
    return res[:, :-total_shit_features]


def true_f(row, pairs=None):
    func_1 = None
    if row[0]:
        func_1 = lambda x: -x
    else:
        func_1 = lambda x: x

    func_2 = None
    if row[1] == 0:
        func_2 = lambda x: -x * x
    if row[1] == 1:
        func_2 = lambda x: x
    else:
        func_2 = lambda x: 2 * x
    return round(
        func_1(func_2(row[2] + row[3]) + row[4] * row[5]) - row[6] * row[7] + 1,
        4,
    )


for i in range(X.shape[1]):
    ys.append(true_f(X[:, i]))

0 (array([0., 1.]), array([25045, 24955]))
1 (array([0., 1., 2.]), array([16940, 16499, 16561]))


In [4]:
X.shape, X[0].copy()[np.newaxis, :].shape

((8, 50000), (1, 50000))

In [5]:
def corrupt_data(X):
    print(X.shape)
    pairs = []
    for i in range(total_redundant_features):
        B = X[i].copy() 
        X = np.concatenate((X,B.reshape(1,-1)), axis=0)
        pairs.append((i, X.shape[0] - 1))
    return X, pairs
    

In [6]:
X_corrupted, pairs = corrupt_data(X)
X_corrupted.shape

(8, 50000)


(12, 50000)

In [7]:
pairs

[(0, 8), (1, 9), (2, 10), (3, 11)]

In [8]:
len(ys), ys[0], true_f(X[:, 0])

(50000, 15.2833, 15.2833)

In [9]:
X_shit = np.random.rand(total_shit_features, 50_000)

In [10]:
X.shape

(8, 50000)

In [11]:
X_shitty = np.concatenate([X_shit, X_corrupted])
X_shitty.shape

(16, 50000)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_shitty.T, ys,test_size=0.2)

In [13]:
def corrupt_test(X_test, pairs):
    for fst, snd in pairs:
        fst += total_shit_features
        snd += total_shit_features
        X_test[:, fst] = X_test[:, fst] * (np.random.rand(X_test.shape[0]) < 0.2)
    return X_test

In [14]:
X_test.shape

(10000, 16)

In [15]:
X_test_corrupted = corrupt_test(X_test, pairs)

In [16]:
mean_squared_error(y_true=y_test, y_pred=np.apply_along_axis(true_f,1, X_test_corrupted)), \
mean_squared_error(y_true=y_test, y_pred=np.apply_along_axis(true_f,1, fix_dataset(X_test_corrupted, pairs))), \
mean_squared_error(y_true=y_train, y_pred=np.apply_along_axis(true_f,1, X_train))

(317.44405840749005, 0.0, 459.6716602708623)

In [17]:
y_train = np.array(y_train)
y_test = np.array(y_test)
np.savetxt('X_train.csv', X_train, delimiter=',', )
np.savetxt('y_train.csv', y_train, delimiter=',')
np.savetxt('X_test.csv', X_test_corrupted, delimiter=',')


In [18]:
test_df = pd.DataFrame(y_test, columns=['y'])

In [19]:
test_df['Usage'] = ['Public'] * len(y_test)
test_df['ID'] = list(range(len(y_test)))

In [20]:
test_df[['ID', 'y', 'Usage']].to_csv('y_test.csv',sep=',', index=False)